In [1]:
import numpy as np

In [72]:
class Task():
    def __init__(self, period, execution_time, deadline ):
        self.period = period
        self.execution_time = execution_time
        self.deadline = deadline
        self.priority = None

    def __repr__(self) -> str:
        return "Task: " +  str(self.period) +" " + str(self.execution_time) + " " + str(self.deadline) + " " + str(self.priority)

class taskset():

    def __init__(self, n, ut, min_period, max_period):
        self.n = n
        self.ut = ut
        self.min_period = min_period
        self.max_period = max_period
        self.tasks = None

    def generate(self):
        self.utilities = self.uunifastdiscard()
        # if not self.utilities:
        #     print("Failed to generate taskset with given parameters")
        #     return False

        self.periods = self.periods_generation()
        self.execution_time = self.execution_times()
        self.deadline = self.task_deadlines()

        self.tasks = []
        for i in range(self.n):
            self.tasks.append(Task(self.periods[i],self.execution_time[i],self.deadline[i]))
        return True
        


    def uunifastdiscard(self,discard_limit = 1000):
        sumu = self.ut
        # for j in range(discard_limit):
        while True:
            utilities = []
            for i in range(self.n-1):
                nextsumu = sumu*(np.random.random()**(1/(self.n-i+1)))
                utilities.append(sumu - nextsumu)
                sumu = nextsumu

            utilities.append(sumu)

            if all(ut <= 1 for ut in utilities):
                return np.array(utilities)
            else:
                sumu = self.ut

        return False
    
    def periods_generation(self):
        periods = np.ceil(np.exp(np.random.uniform(low=np.log(self.min_period), high=np.log(self.max_period),size=(self.n))))
        return periods
    
    def execution_times(self):
        return np.ceil(self.utilities * self.periods)
    
    def task_deadlines(self):
        deadline = []
        for i in range(len(self.execution_time)):
            deadline.append(np.random.uniform(low = self.execution_time[i] , high = self.periods[i]))

        return np.floor(np.array(deadline))

            


In [84]:
cores = 2
set = taskset(cores*5,1,10,100)

set.generate()
print(set.tasks)

[Task: 46.0 4.0 29.0 None, Task: 35.0 7.0 21.0 None, Task: 15.0 1.0 12.0 None, Task: 20.0 3.0 6.0 None, Task: 25.0 3.0 15.0 None, Task: 12.0 1.0 2.0 None, Task: 17.0 2.0 9.0 None, Task: 60.0 4.0 28.0 None, Task: 89.0 2.0 2.0 None, Task: 19.0 5.0 14.0 None]


In [116]:
def DMPO(set):
    set.sort(key = lambda x:x.deadline)

In [119]:
DMPO(set.tasks)
print(set.tasks)

[Task: 48.0 1.0 6.0 None, Task: 19.0 2.0 7.0 None, Task: 21.0 1.0 10.0 None, Task: 19.0 1.0 10.0 None, Task: 37.0 9.0 15.0 None, Task: 25.0 1.0 17.0 None, Task: 32.0 11.0 22.0 None, Task: 31.0 3.0 29.0 0, Task: 46.0 7.0 29.0 None, Task: 81.0 4.0 36.0 None]


In [122]:
deadline_analysis(set.tasks,2)

TypeError: 'Task' object is not subscriptable

In [159]:
def OPA(taskset, cores, S_test):
    N = len(taskset)

    assigned_tasks = []

    taskset_copy = taskset.copy()

    priority_assigned = [False] * N

    for i in range(N-1,-1,-1):

        for j,task in enumerate(taskset_copy):

            if task.priority == None:
                if(S_test(assigned_tasks + [task],cores)):
                    task.priority = i
                    priority_assigned[j] = True
                    assigned_tasks.append(task)
                    break
        else:
            return taskset_copy,i
    
    return taskset_copy,i

In [120]:
def deadline_analysis(ptaskset,m):    #taskset = {period,execution time,deadline}
    num_tasks = len(ptaskset)
    for i in range(num_tasks):
        total_workload = 0
        ei,di = ptaskset[i][1],ptaskset[i][2]
        l = di
        for j in range(i):
            pj,ej,dj = ptaskset[j][0],ptaskset[j][1],ptaskset[j][2]
            max_jobs = np.floor(( l + dj - ej )/pj)
            workload_j = max_jobs*ej + min(ej,l + dj - ej - max_jobs*pj)
            workload_j = min(workload_j,di - ei + 1)
            total_workload += workload_j

        thresh = ei + np.floor(total_workload/m)
        if di < thresh:
            return 0
    return 1

In [125]:
def deadline_analysis_OPA(ptaskset,m):    #taskset = {period,execution time,deadline}
    Ck = ptaskset[-1].execution_time
    Dk = ptaskset[-1].deadline
    l = Dk
    total_workload = 0

    for task in ptaskset[:-1]:
        Ti,Ci,Di = task.period,task.execution_time,task.deadline
        max_jobs = np.floor(( l + Di - Ci )/Ti)
        workload_task = max_jobs*Ci + min(Ci,l + Di - Ci - max_jobs*Ti)
        workload_task = min(workload_task,Dk - Ck + 1)
        total_workload += workload_task

        thresh = Ck + np.floor(total_workload/m)
        if Dk < thresh:
            return 0
    return 1

In [160]:
cores = 2
set = taskset(cores*5,0.2,10,100)

set.generate()
print(set.tasks)

[Task: 28.0 1.0 14.0 None, Task: 40.0 1.0 15.0 None, Task: 74.0 1.0 28.0 None, Task: 14.0 1.0 6.0 None, Task: 16.0 1.0 12.0 None, Task: 73.0 1.0 46.0 None, Task: 19.0 1.0 6.0 None, Task: 22.0 1.0 20.0 None, Task: 69.0 1.0 61.0 None, Task: 27.0 3.0 12.0 None]


In [161]:
ordered_tasks,i = OPA(set.tasks, cores, deadline_analysis_OPA)
print(ordered_tasks)
print(i)

[Task: 28.0 1.0 14.0 9, Task: 40.0 1.0 15.0 8, Task: 74.0 1.0 28.0 7, Task: 14.0 1.0 6.0 6, Task: 16.0 1.0 12.0 5, Task: 73.0 1.0 46.0 4, Task: 19.0 1.0 6.0 3, Task: 22.0 1.0 20.0 2, Task: 69.0 1.0 61.0 1, Task: 27.0 3.0 12.0 0]
0


8
